In [ ]:
import serial                  # used to setup connection between python and arduino
import matplotlib.pyplot as plt
import librosa
import librosa.display
import IPython.display as ipd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense
import seaborn as sns
x=[]            
y=[]

# Code to visualize data

In [ ]:
data=[]                     
ar_data=serial.Serial("COM10",250000) # getting data from arduino
plt.ion()
while True:
    while (ar_data.inWaiting()==0):
        pass
    try:
        ar_str=ar_data.readline() # reading data
        data.append(float(ar_str)) # storing in a list
        plt.cla()                 # following lines plots the data
        plt.xlim(0,256)
        plt.plot(data)
        plt.show()
        plt.pause(0.001)
    except KeyboardInterrupt:   # stops plotting with keyboard interuption
        break

# Creating dataset to train classifier  

In [ ]:
datatrain=np.linspace(0,100,100)          # initializing training data with fake inputs later we will update it.
ar_data_train=serial.Serial("COM10",250000) # getting data from arduino
while True:
    while (ar_data_train.inWaiting()==0): 
        pass
    try:
        ar_train_str=ar_data_train.readline() # reading data
        datatrain=np.append(datatrain,float(ar_train_str)) #updating datatrain  
        datatrain=np.delete(datatrain,[0]) 
        mfccs=librosa.feature.mfcc(datatrain,n_mfcc=13,sr=100) # we are taking 100 elements at a time and extracting 13 mfccs
        x.append(mfccs)
        y.append(1) # here value will be changed while training based on the present configuration of fingers 0 for all open 1 for all close.
    except KeyboardInterrupt:
        break
        
        

# Creating Pandas dataset.

In [ ]:
from array import *
def array_list(array_num): 
    num_list = array_num.tolist() # list 
    return num_list
finalx=[]
n=0
while n<=11403:    #iterating through whole data points 
    array_list(x[n])
    l=[]
    for i in x[n]:
        l.append(i[0])  #making another list finalx so that we can use it to make pandas dataset.
    finalx.append(l)
    n=n+1
"""The above lines of code makes list finalx from list x later from finalx we can create pandas dataset we can't
use list x directly for creating data set because it's a list of lists"""

In [ ]:
df=pd.DataFrame(finalx) # creating pandas data set
df["y"]=y

df.to_csv('D:\MATRA\CFI\Handyman\eeg_data_set1.csv') # saving .csv file

# Training

In [ ]:
predictors=df.iloc[:,0:13]  #seperating input(MFCCs) from dataset.

response=df.iloc[:,13]      #seperating output from dataset.

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(predictors,response,test_size=0.2) # spitting dataset into training and testing parts

In [ ]:
kerasmodel = Sequential()
kerasmodel.add(Dense(13,input_dim=13,activation='relu')) #NN input layer one with 13 neurons since we have 13 MFCCs.
kerasmodel.add(Dense(2,activation='relu'))               #Hidden Layer with 2 neuron using relu activation function
kerasmodel.add(Dense(1,activation='sigmoid'))            #output layer with sigmod fucntion which return 0 or 1.

In [ ]:
kerasmodel.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy']) #specifing loss function,optimizer.

In [ ]:
kerasmodel.fit(X_train,y_train,epochs=2000) #initiating training


# Testing

In [ ]:

_,accuracy=kerasmodel.evaluate(X_test,y_test) #testing accuracy

In [ ]:
accuracy

In [ ]:
datatrain=np.linspace(0,100,100)
ar_data_test=serial.Serial("COM10",250000,timeout=2)
while True:
    while (ar_data_train.inWaiting()==0):
        pass
    try:
        ar_train_str=ar_data_test.readline()
        datatrain=np.append(datatrain,float(ar_train_str))
        datatrain=np.delete(datatrain,[0])
        mfccs=librosa.feature.mfcc(datatrain,n_mfcc=13,sr=100)
        lmfcc=[]
        n=0
        while n<=12:   
            array_list(mfccs[n])
            l=[]
            for i in mfccs[n]:
                l.append(i)
                lmfcc.append(i)
            n=n+1
        test=pd.DataFrame(list([lmfcc])) 
        a=kerasmodel.predict_classes(test)
        print(a[0][0])
    except KeyboardInterrupt:
        break
        
"""We have collected the data and trained a ML model, Now it’s time to test it in real time.
We will start with connecting electrodes to scalp ,after running Arduino code in Arduino IDE
we get the data through AD8232 ECG module to Arduino and Arduino sends data to serial ports 
where we use python to get data from these ports(line 2 in the code) . Later we read the data
and extract MFCC using librosa and store MFCCs in list ‘mfcc’ after this we need make another
list ‘lmfcc’ in order to create a pandas data set for testing. After creating pandas dataset 
we test this dataset with our saved ML model and print the final output i.e 0 or 1.
"""